In [1]:
%load_ext autoreload
%autoreload 2
import gc
import re
import pandas as pd
from __future__ import print_function, division
import numpy as np
pd.set_option('max_columns', 200)
pd.set_option('max_rows', 200)
import os
import sys
HOME = os.path.expanduser("~")
sys.path.append(f'{HOME}/kaggle/data_analysis/library')
import utils
from utils import get_categorical_features, get_numeric_features, reduce_mem_usage, elo_save_feature
from preprocessing import get_dummies
import datetime
import glob

from tqdm import tqdm
import time
import sys
from joblib import Parallel, delayed

key = 'card_id'
target = 'target'
ignore_list = [key, target, 'merchant_id', 'first_avtive_month']

In [ ]:
#========================================================================
# Dataset Load 
fam = 'first_active_month'
term_name = 'hist_regist_term'
base = utils.read_pkl_gzip('../input/base_no_out_clf.gz')

win_path_list = glob.glob('../features/4_winner/*.gz')
feature_list = utils.parallel_load_data(path_list=win_path_list)
df_feat = pd.concat(feature_list, axis=1)
df_feat = pd.concat([base[[key, target, fam, term_name]], df_feat], axis=1)
df_feat.head()
#========================================================================

In [45]:
# term = term_list[1]
# month = fam_list[0]
# tmp = df_feat[df_feat[term_name]==term]

In [52]:
#========================================================================
# Lift FAM * hist_regist_term
term_list = sorted(list(base[term_name].value_counts().index))
feat_cols = [col for col in df_feat.columns if col.count('@')]

from joblib import Parallel, delayed
def make_df_fam(args):
    term = args[0]
    month = args[1]
    tmp_list = []
    for col in feat_cols:
        tmp = df_feat[df_feat[term_name]==term][df_feat[fam]==month].reset_index()
        tmp = tmp[[key, col]]
        base_avg = tmp[col].mean()
        tmp.set_index(key, inplace=True)
        tmp[col] /= base_avg
        tmp.rename(columns={col:f'lift_FAM_{term_name}_{col}'}, inplace=True)
        tmp_list.append(tmp)
    tmp_fam = pd.concat(tmp_list, axis=1)
    return tmp_fam

df_fam_list = []
df_term_list = []
for term in tqdm(term_list):
    length = len(df_feat[df_feat[term_name]==term][key].value_counts().index)
    if length<3:
        continue
#     for month in tqdm(fam_list):
#         tmp_list = []
#         for col in feat_cols:
#             tmp = df_feat[df_feat[term_name]==term][df_feat[fam]==month].reset_index()
#             tmp = tmp[[key, col]]
#             base_avg = tmp[col].mean()
#             tmp.set_index(key, inplace=True)
#             tmp[col] /= base_avg
#             tmp.rename(columns={col:f'lift_FAM_{term_name}_{col}'}, inplace=True)
#             tmp_list.append(tmp)

#         tmp_fam = pd.concat(tmp_list, axis=1)
#         df_fam_list.append(tmp_fam)
    fam_list = [str(f)[:7] for f in list(base[fam].value_counts().index)]
    arg_list = []
    for f in fam_list:
        arg_list.append([term, f])
    df_fam_list = Parallel(n_jobs=-1)([delayed(make_df_fam)(args) for args in arg_list])
    df_fam = pd.concat(df_fam_list, axis=0)
    df_term_list.append(df_fam)
df_lift = pd.concat(df_term_list, axis=0)
df_lift.fillna(-1, inplace=True)
print(df_lift.shape)

tmp_base = utils.read_pkl_gzip('../input/base_no_out_clf.gz').set_index(key)['target'].to_frame()
df_lift = tmp_base.join(df_lift)
del tmp_base
gc.collect()
df_lift.head()
#========================================================================






  0%|          | 0/25 [00:00<?, ?it/s]




  8%|▊         | 2/25 [00:08<01:38,  4.29s/it]




 12%|█▏        | 3/25 [00:17<02:05,  5.72s/it]




 16%|█▌        | 4/25 [00:29<02:38,  7.53s/it]




 20%|██        | 5/25 [00:41<02:57,  8.87s/it]




 24%|██▍       | 6/25 [00:53<03:05,  9.75s/it]




 28%|██▊       | 7/25 [01:05<03:07, 10.39s/it]




 32%|███▏      | 8/25 [01:16<03:02, 10.75s/it]




 36%|███▌      | 9/25 [01:28<02:56, 11.01s/it]




 40%|████      | 10/25 [01:39<02:47, 11.17s/it]




 44%|████▍     | 11/25 [01:51<02:36, 11.19s/it]




 48%|████▊     | 12/25 [02:02<02:24, 11.14s/it]




 52%|█████▏    | 13/25 [02:13<02:13, 11.12s/it]




 56%|█████▌    | 14/25 [02:24<02:01, 11.06s/it]




 60%|██████    | 15/25 [02:35<01:50, 11.05s/it]




 64%|██████▍   | 16/25 [02:46<01:39, 11.02s/it]




 68%|██████▊   | 17/25 [02:56<01:27, 10.88s/it]




 72%|███████▏  | 18/25 [03:07<01:15, 10.74s/it]




 76%|███████▌  | 19/25 [03:17<01:03, 10.60s/it]




 80%|████████  | 20/25 [

(325539, 50)


,target,lift_FAM_hist_regist_term_252_rad_auth1_all_top-M1_merchant_flag-2_-4-plag-5_-10_purchase_amount_new_sum@,lift_FAM_hist_regist_term_244_rad_auth1_all_top-M2_merchant_flag0_-2-plag-3_-8_purchase_amount_new_sum@,lift_FAM_hist_regist_term_241_rad_auth1_all_flag-2_-3-plag-4_-9_purchase_amount_new_sum_per_installments_sum_mean@,lift_FAM_hist_regist_term_242_rad_freq_2Mover_auth1_all_flag-1_-2-plag-3_-5_purchase_amount_new_sum_per_installments_sum_mean@,lift_FAM_hist_regist_term_241_rad_auth1_all_flag-1_-2-plag-3_-4_purchase_amount_new_sum_mean@,lift_FAM_hist_regist_term_241_rad_auth1_all_flag-1_-2-plag-3_-8_purchase_amount_new_sum_mean@,lift_FAM_hist_regist_term_241_rad_auth1_all_flag0_-2-plag-3_-8_purchase_amount_new_sum_per_installments_sum_max@,lift_FAM_hist_regist_term_241_rad_auth1_all_flag0_-1-plag-2_-7_installments_sum_mean@,lift_FAM_hist_regist_term_245_rad_min_Mlag-6_auth1_all_top-M2_merchant_flag0_-2-plag-3_-5_purchase_amount_new_sum_per_installments_sum@,lift_FAM_hist_regist_term_245_rad_min_Mlag-6_auth1_all_top-M2_0_merchant_flag-1_-1-plag-2_-7_purchase_amount_new_sum@,lift_FAM_hist_regist_term_241_rad_auth1_all_flag-2_-4-plag-5_-10_installments_mean_mean@,lift_FAM_hist_regist_term_252_rad_auth1_all_top-M1_merchant_flag-2_-3-plag-4_-5_purchase_amount_new_sum_per_installments_sum@,lift_FAM_hist_regist_term_242_rad_freq_2Mover_auth1_all_flag-2_-3-plag-4_-9_purchase_amount_new_sum_per_installments_sum_mean@,lift_FAM_hist_regist_term_245_rad_min_Mlag-6_auth1_all_top-M1_merchant_flag-1_-3-plag-4_-6_purchase_amount_new_sum_per_installments_sum@,lift_FAM_hist_regist_term_242_rad_freq_2Mover_auth1_all_flag0_-1-plag-2_-7_purchase_amount_new_sum_mean@,lift_FAM_hist_regist_term_241_rad_auth1_all_flag-2_-4-plag-5_-7_purchase_amount_new_sum_per_installments_sum_mean@,lift_FAM_hist_regist_term_241_rad_auth1_all_flag-2_-4-plag-5_-10_purchase_amount_new_sum_max@,lift_FAM_hist_regist_term_241_rad_auth1_all_flag-2_-4-plag-5_-10_purchase_amount_new_sum_per_installments_sum_mean@,lift_FAM_hist_regist_term_241_rad_auth1_all_flag-1_-3-plag-4_-9_purchase_amount_new_sum_per_installments_sum_mean@,lift_FAM_hist_regist_term_241_rad_auth1_all_flag-2_-4-plag-5_-10_purchase_amount_new_sum_per_installments_sum_max@,lift_FAM_hist_regist_term_245_rad_min_Mlag-6_auth1_all_top-M2_merchant_flag0_-1-plag-2_-3_purchase_amount_new_sum@,lift_FAM_hist_regist_term_242_rad_freq_2Mover_auth1_all_flag0_-2-plag-3_-8_installments_sum_mean@,lift_FAM_hist_regist_term_241_rad_auth1_all_flag-2_-3-plag-4_-6_purchase_amount_new_sum_max@,lift_FAM_hist_regist_term_244_rad_auth1_all_top-M1_merchant_flag0_-2-plag-3_-8_purchase_amount_new_sum_per_installments_sum@,lift_FAM_hist_regist_term_242_rad_freq_2Mover_auth1_all_flag-2_-4-plag-5_-10_purchase_amount_new_sum_mean@,lift_FAM_hist_regist_term_244_rad_auth1_all_top-M1_merchant_flag0_-2-plag-3_-5_purchase_amount_new_sum_per_installments_sum_sum@,lift_FAM_hist_regist_term_241_rad_auth1_all_flag-1_-3-plag-4_-9_installments_max_mean@,lift_FAM_hist_regist_term_241_rad_auth1_all_flag-1_-2-plag-3_-8_installments_sum_mean@,lift_FAM_hist_regist_term_242_rad_freq_2Mover_auth1_all_flag-1_-3-plag-4_-9_purchase_amount_new_sum_max@,lift_FAM_hist_regist_term_252_rad_auth1_all_top-M3_merchant_flag-1_-3-plag-4_-9_purchase_amount_new_sum_per_installments_sum@,lift_FAM_hist_regist_term_242_rad_freq_2Mover_auth1_all_flag-2_-4-plag-5_-10_purchase_amount_new_sum_per_installments_sum_mean@,lift_FAM_hist_regist_term_241_rad_auth1_all_flag0_-1-plag-2_-7_purchase_amount_new_sum_max@,lift_FAM_hist_regist_term_242_rad_freq_2Mover_auth1_all_flag-1_-3-plag-4_-9_purchase_amount_new_sum_per_installments_sum_mean@,lift_FAM_hist_regist_term_252_rad_auth1_all_top-M2_merchant_flag-1_-3-plag-4_-9_purchase_amount_new_sum_per_installments_sum@,lift_FAM_hist_regist_term_244_rad_auth1_all_top-M1_merchant_flag-1_-3-plag-4_-6_purchase_amount_new_sum_per_installments_sum_sum@,lift_FAM_hist_regist_term_241_rad_auth1_all_flag-2_-4-plag-5_-10_purchase_amount_new_

In [56]:
prefix = '255_rad'
#========================================================================
# Save Feature
elo_save_feature(df_feat=df_lift, prefix=prefix)
#========================================================================
print('Complete!')

Complete!


In [57]:
#========================================================================
# Lift FAM * hist_regist_term
# fam_list = sorted(list(base[fam].value_counts().index))
term_list = sorted(list(base[term_name].value_counts().index))
feat_cols = [col for col in df_feat.columns if col.count('@')]

def make_df_term(term):
    tmp_list = []
    for col in feat_cols:
        tmp = df_feat[df_feat[term_name]==term].reset_index()
        tmp = tmp[[key, col]]
        base_avg = tmp[col].mean()
        tmp.set_index(key, inplace=True)
        tmp[col] /= base_avg
        tmp.rename(columns={col:f'lift_{term_name}_{col}'}, inplace=True)
        tmp_list.append(tmp)
    tmp_term = pd.concat(tmp_list, axis=1)
    return tmp_term

df_term_list = Parallel(n_jobs=-1)([delayed(make_df_term)(term) for term in term_list])
df_term_lift = pd.concat(df_term_list, axis=0)
    
tmp_base = utils.read_pkl_gzip('../input/base_no_out_clf.gz').set_index(key)['target'].to_frame()
df_term_lift = tmp_base.join(df_term_lift)
print(df_term_lift.shape)
del tmp_base
gc.collect()
df_term_lift.head(100)
#========================================================================

(325540, 51)


,target,lift_hist_regist_term_252_rad_auth1_all_top-M1_merchant_flag-2_-4-plag-5_-10_purchase_amount_new_sum@,lift_hist_regist_term_244_rad_auth1_all_top-M2_merchant_flag0_-2-plag-3_-8_purchase_amount_new_sum@,lift_hist_regist_term_241_rad_auth1_all_flag-2_-3-plag-4_-9_purchase_amount_new_sum_per_installments_sum_mean@,lift_hist_regist_term_242_rad_freq_2Mover_auth1_all_flag-1_-2-plag-3_-5_purchase_amount_new_sum_per_installments_sum_mean@,lift_hist_regist_term_241_rad_auth1_all_flag-1_-2-plag-3_-4_purchase_amount_new_sum_mean@,lift_hist_regist_term_241_rad_auth1_all_flag-1_-2-plag-3_-8_purchase_amount_new_sum_mean@,lift_hist_regist_term_241_rad_auth1_all_flag0_-2-plag-3_-8_purchase_amount_new_sum_per_installments_sum_max@,lift_hist_regist_term_241_rad_auth1_all_flag0_-1-plag-2_-7_installments_sum_mean@,lift_hist_regist_term_245_rad_min_Mlag-6_auth1_all_top-M2_merchant_flag0_-2-plag-3_-5_purchase_amount_new_sum_per_installments_sum@,lift_hist_regist_term_245_rad_min_Mlag-6_auth1_all_top-M2_0_merchant_flag-1_-1-plag-2_-7_purchase_amount_new_sum@,lift_hist_regist_term_241_rad_auth1_all_flag-2_-4-plag-5_-10_installments_mean_mean@,lift_hist_regist_term_252_rad_auth1_all_top-M1_merchant_flag-2_-3-plag-4_-5_purchase_amount_new_sum_per_installments_sum@,lift_hist_regist_term_242_rad_freq_2Mover_auth1_all_flag-2_-3-plag-4_-9_purchase_amount_new_sum_per_installments_sum_mean@,lift_hist_regist_term_245_rad_min_Mlag-6_auth1_all_top-M1_merchant_flag-1_-3-plag-4_-6_purchase_amount_new_sum_per_installments_sum@,lift_hist_regist_term_242_rad_freq_2Mover_auth1_all_flag0_-1-plag-2_-7_purchase_amount_new_sum_mean@,lift_hist_regist_term_241_rad_auth1_all_flag-2_-4-plag-5_-7_purchase_amount_new_sum_per_installments_sum_mean@,lift_hist_regist_term_241_rad_auth1_all_flag-2_-4-plag-5_-10_purchase_amount_new_sum_max@,lift_hist_regist_term_241_rad_auth1_all_flag-2_-4-plag-5_-10_purchase_amount_new_sum_per_installments_sum_mean@,lift_hist_regist_term_241_rad_auth1_all_flag-1_-3-plag-4_-9_purchase_amount_new_sum_per_installments_sum_mean@,lift_hist_regist_term_241_rad_auth1_all_flag-2_-4-plag-5_-10_purchase_amount_new_sum_per_installments_sum_max@,lift_hist_regist_term_245_rad_min_Mlag-6_auth1_all_top-M2_merchant_flag0_-1-plag-2_-3_purchase_amount_new_sum@,lift_hist_regist_term_242_rad_freq_2Mover_auth1_all_flag0_-2-plag-3_-8_installments_sum_mean@,lift_hist_regist_term_241_rad_auth1_all_flag-2_-3-plag-4_-6_purchase_amount_new_sum_max@,lift_hist_regist_term_244_rad_auth1_all_top-M1_merchant_flag0_-2-plag-3_-8_purchase_amount_new_sum_per_installments_sum@,lift_hist_regist_term_242_rad_freq_2Mover_auth1_all_flag-2_-4-plag-5_-10_purchase_amount_new_sum_mean@,lift_hist_regist_term_244_rad_auth1_all_top-M1_merchant_flag0_-2-plag-3_-5_purchase_amount_new_sum_per_installments_sum_sum@,lift_hist_regist_term_241_rad_auth1_all_flag-1_-3-plag-4_-9_installments_max_mean@,lift_hist_regist_term_241_rad_auth1_all_flag-1_-2-plag-3_-8_installments_sum_mean@,lift_hist_regist_term_242_rad_freq_2Mover_auth1_all_flag-1_-3-plag-4_-9_purchase_amount_new_sum_max@,lift_hist_regist_term_252_rad_auth1_all_top-M3_merchant_flag-1_-3-plag-4_-9_purchase_amount_new_sum_per_installments_sum@,lift_hist_regist_term_242_rad_freq_2Mover_auth1_all_flag-2_-4-plag-5_-10_purchase_amount_new_sum_per_installments_sum_mean@,lift_hist_regist_term_241_rad_auth1_all_flag0_-1-plag-2_-7_purchase_amount_new_sum_max@,lift_hist_regist_term_242_rad_freq_2Mover_auth1_all_flag-1_-3-plag-4_-9_purchase_amount_new_sum_per_installments_sum_mean@,lift_hist_regist_term_252_rad_auth1_all_top-M2_merchant_flag-1_-3-plag-4_-9_purchase_amount_new_sum_per_installments_sum@,lift_hist_regist_term_244_rad_auth1_all_top-M1_merchant_flag-1_-3-plag-4_-6_purchase_amount_new_sum_per_installments_sum_sum@,lift_hist_regist_term_241_rad_auth1_all_flag-2_-4-plag-5_-10_purchase_amount_new_sum_mean@,lift_hist_regist_term_241_rad_auth1_all_flag0_-2-plag-3_-5_purchase_amount_new_sum_per_installments_sum_max@,lift_hist_regist_term_241

In [58]:
prefix = '256_rad'
#========================================================================
# Save Feature
elo_save_feature(df_feat=df_term_lift, prefix=prefix)
#========================================================================
print('Complete!')

Complete!


In [59]:
del df_lift, df_term_lift
gc.collect()

7